# Cvičenie 4: Policy Iteration

Na minulom cvičení ste videli, ako sa dajú vyriešiť Markovovské rozhodovacie procesy pomocou Bellmanových rovníc. Od tohto týždňa sa pozrieme na rôzne metódy učenia posilňovaním, pričom ich budeme aplikovať na rovnaký gridworld problém pre porovnávanie. Prvá metóda bude práve Policy Iteration.

## Policy Iteration

Policy Iteration pozostáva z iteratívnej aplikácie dvoch krokov, a to vyhodnocovanie (*policy evaluation*) a zlepšovanie (*policy improvement*). Pseudokód algoritmu nájdete na obrázku nižšie.

<img src="lab04/policy_iteration.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Prediskutujte význam jednotlivých operácií a krokov a následne aplikujte algoritmus na ukážkový príklad gridworldu. Ako ukážku, nižšie nájdete vypracované kroky pre jeden stav.

<img src="lab04/gridworld_mdp.jpg" width="600">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v strede sveta. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra ukončí. Pre ostatné kroky dostane agent odmenu -1.

Dolná a pravá časť sveta je úplne deterministická, pričom na pozíciach uznačených svetlomodrou farbou fúka silný vietor, ktorý môže agenta posunúť na východ aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na východ je $0.4$. Ak agent vyberie pohyb na východ, určite sa tam dostane. Ak sa pohybuje na západ, ostane na svojej pôvodnej pozícii (vietor a jeho pohyb sa rušia).

Discount factor $\gamma = 0.8$ a $\theta = 0.1$. Začneme s politikou $\pi(s) = N \ \ \forall \ \ s \in \mathcal{S}$, teda na každej pozícii sa vyberie pohyb na sever.

### 1. Policy Evaluation

Pre stav $s_{11}$ (prvý riadok, prvý stĺpec) vieme vypočítať hodnotu stavu tak, že berieme do úvahy všetky možné prechody pri výbere akcie $N$. V našom svete pre túto nedeterministickú pozíciu existujú dva možné prechody: $s_{11}$ a $s_{12}$. Na základe pravdepodobností prechodov vieme vypočítať hodnotu tohto stavu:

$V(s_{11}) = p \left ( s_{11}, r_{s11} | s_{11}, \pi(s_{11}) \right ) \cdot (r_{s11} + \gamma \cdot V(s_{11})) + p \left ( s_{12}, r_{s12} | s_{11}, \pi(s_{11}) \right ) \cdot (r_{s12} + \gamma \cdot V(s_{12})) $

$V(s_{11}) = 0.6 \cdot (-1 + 0.8 \cdot 0) + 0.4 \cdot (-1 + 0.8 \cdot 0) $

$V(s_{11}) = -1$

Zmena odhadu pre $V(s_{11})$ je $\Delta V(s_{11}) = |0 - (-1)| = 1$, čo je väčšie ako $\theta$, takže už teraz je zrejmé, že vyhodnocovanie si budeme musieť zopakovať.

**Úloha:** Vypočítajte hodnotu ďalších stavov (okrem $s_{13}$ a $s_{22}$). Použite pritom inplace prístup.

**Úloha:** Vypočítajte hodnotu ďalších stavov na základe nasledujúcich hodnôt, ktoré by ste dostali po niekoľkých iteráciách:

$V(s_{11}) = 1.909$

$V(s_{12}) = 6.5$

$V(s_{13}) = 0$

$V(s_{21}) = -3.684$

$V(s_{22}) = 0$

$V(s_{23}) = 10$

$V(s_{31}) = 3.947$

$V(s_{32}) = -10$

$V(s_{33}) = 7$

### 2. Policy Improvement

Začiatočnú politiku $\pi_0$ definujeme ľubovoľne, v našom prípade to bola politika posunu na sever v každom stave. Ak už máme vypočítane hodnoty stavov, tieto hodnoty vieme použiť pri zlepšovaní politiky. Pre každú akciu vypočítame očakávanú dlhodobú odmenu podľa vzorca:

$q(s, a) = \sum_{s', r} p(s', r|s, a) \cdot (r + \gamma \cdot V(s'))$

Teda pre stav $s_{11}$:

$q(s_{11}, N) = p(s_{11}, r_{s11} | s_{11}, N) \cdot (r_{s11} + \gamma \cdot V(s_{11})) + p(s_{12}, r_{s12} | s_{11}, N) \cdot  (r_{s12} + \gamma \cdot V(s_{12})) = 0.6 \cdot (-1 + 0.8 \cdot 1.996) + 0.4 \cdot (-1 + 0.8 \cdot 6.52) = 2.044 $

$q(s_{11}, E) = p(s_{12}, r_{s12} | s_{11}, E) \cdot (r_{s12} + \gamma \cdot V(s_{12})) = 1.0 \cdot (-1 + 0.8 \cdot 6.52) = 4.216 $

$q(s_{11}, S) = p(s_{21}, r_{s21} | s_{11}, S) \cdot (r_{s21} + \gamma \cdot V(s_{21})) + p(s_{12}, r_{s12} | s_{11}, S) \cdot  (r_{s12} + \gamma \cdot V(s_{12})) = 0.6 \cdot (-1 + 0.8 \cdot (-3.684)) + 0.4 \cdot (-1 + 0.8 \cdot 6.52) = -0.6616 $

$q(s_{11}, W) = p(s_{11}, r_{s11} | s_{11}, W) \cdot (r_{s11} + \gamma \cdot V(s_{11})) = 1.0 \cdot (-1 + 0.8 \cdot 1.996) = 0.597 $

pre jednotlivé akcie. Následne aplikujeme funkciu $argmax$, ktorá nám vráti akciu, pri ktorej je hodnota maximálna:

$\pi(s_{11}) = argmax_a(q(s_{11}, N), q(s_{11}, E), q(s_{11}, S), q(s_{11}, W)) = E$

**Úloha:** Vypočítajte všetky očakávané odmeny pre dvojice stav-akcia (okrem stavov $s_{13}$ a $s_{22}$) a na základe hodnôt aktualizujte politiku $\pi$.

## Implementácia

Vaše výpočty overte na [ukážkovej implementácii metódy Policy Iteration](lab04/policy_iteration.py) a spozorujte, ako sa metóda dopracuje k optimálnej politike.

**Domáca úloha:** Implementáciu opravte tak, aby namiesto inplace stratégie používala sweep stratégiu.